In [36]:
import os
import pandas as pd
import matplotlib.pyplot as plt

if os.getcwd().split('/')[-1] == 'mobility':
    os.chdir('..')
from importlib import reload
import countries
reload(countries)
from countries import FROM_COUNTRY

df = pd.read_csv('dist/static/mobility/world.csv.gz')
df

,country_geoid,country,region,category,page,date,change,value
0,AE,United Arab Emirates,total,grocery/pharmacy,0,2020-02-16,-26,4.536738
1,AE,United Arab Emirates,total,grocery/pharmacy,0,2020-02-17,-26,0.672349
2,AE,United Arab Emirates,total,grocery/pharmacy,0,2020-02-18,-26,1.132972
3,AE,United Arab Emirates,total,grocery/pharmacy,0,2020-02-19,-26,0.363650
4,AE,United Arab Emirates,total,grocery/pharmacy,0,2020-02-20,-26,0.851749
...,...,...,...,...,...,...,...,...
207650,ZW,Zimbabwe,total,workplace,1,2020-04-01,-36,-70.065087
207651,ZW,Zimbabwe,total,workplace,1,2020-04-02,-36,-70.249640
207652,ZW,Zimbabwe,total,workplace,1,2020-04-03,-36,-67.565660
207653,ZW,Zimbabwe,total,workplace,1,2020-04-04,-36,-55.972998


In [34]:
pd.read_csv('mobility/world(1).csv.gz')

,country_geoid,country,region,category,page,date,change,value
0,AE,United Arab Emirates,total,grocery/pharmacy,0,2020-02-16,-26,4.536738
1,AE,United Arab Emirates,total,grocery/pharmacy,0,2020-02-17,-26,0.672349
2,AE,United Arab Emirates,total,grocery/pharmacy,0,2020-02-18,-26,1.132972
3,AE,United Arab Emirates,total,grocery/pharmacy,0,2020-02-19,-26,0.363650
4,AE,United Arab Emirates,total,grocery/pharmacy,0,2020-02-20,-26,0.851749
...,...,...,...,...,...,...,...,...
207650,ZW,Zimbabwe,total,workplace,1,2020-04-01,-36,-70.065087
207651,ZW,Zimbabwe,total,workplace,1,2020-04-02,-36,-70.249640
207652,ZW,Zimbabwe,total,workplace,1,2020-04-03,-36,-67.565660
207653,ZW,Zimbabwe,total,workplace,1,2020-04-04,-36,-55.972998


In [19]:
data = df.query('country == "Spain" and region == "Cantabria" and category == "workplace"')
alt.Chart().mark_line(point=True).encode(x='date:T', y='value:Q')
data.head(10)

,country_geoid,country,region,category,page,date,change,value
55183,ES,Spain,Cantabria,workplace,5,2020-02-16,-61,-0.571793
55184,ES,Spain,Cantabria,workplace,5,2020-02-17,-61,2.173386
55185,ES,Spain,Cantabria,workplace,5,2020-02-18,-61,3.047330
55186,ES,Spain,Cantabria,workplace,5,2020-02-19,-61,3.035856
55187,ES,Spain,Cantabria,workplace,5,2020-02-20,-61,2.289083
55188,ES,Spain,Cantabria,workplace,5,2020-02-21,-61,2.029003
55189,ES,Spain,Cantabria,workplace,5,2020-02-22,-61,-0.316494
55190,ES,Spain,Cantabria,workplace,5,2020-02-24,-64,-13.356961
55191,ES,Spain,Cantabria,workplace,5,2020-02-25,-64,-14.103843
55192,ES,Spain,Cantabria,workplace,5,2020-02-26,-64,-12.886100


In [27]:
print(*df.query("country == 'Switzerland'").region.unique(), sep='\n')

Aargau
Appenzell Ausserrhoden
Basel City
Basel-Landschaft
Canton of Bern
Grisons
Jura
Uri
Valais
Vaud
Zurich
total


In [2]:
import altair as alt

# alt.themes.enable('dark')

country = "France"
category = "workplace"
subset = df.query('country == @country and category == @category')
chart_data = subset.groupby(['date'])['value'].mean().reset_index()
(alt.Chart(chart_data)
 .mark_line(point=True)
 .encode(x='date', y='value')
 +
 alt.Chart(subset.query('region == "total"'))
 .mark_line(point=True)
 .encode(x='date', y='value')
)

alt.LayerChart(...)

In [27]:
(alt.Chart(subset.reset_index())
 .mark_line(point=True)
 .encode(x='date', y='value', color='region', tooltip=['region', 'date', 'value'])
 .properties(width=700)
)

In [28]:
fr = df.query('country == "France" and region == "total"')
alt.Chart(fr).mark_line(point=True).encode(x='date:T', y='value', color='category')

In [29]:
df = df.assign(country_code=df['country'].map(FROM_COUNTRY))
df

,country_geoid,country,region,category,page,date,change,value,country_code
0,AE,United Arab Emirates,total,grocery/pharmacy,0,2020-02-16,-26,4.536738,784
1,AE,United Arab Emirates,total,grocery/pharmacy,0,2020-02-17,-26,0.672349,784
2,AE,United Arab Emirates,total,grocery/pharmacy,0,2020-02-18,-26,1.132972,784
3,AE,United Arab Emirates,total,grocery/pharmacy,0,2020-02-19,-26,0.363650,784
4,AE,United Arab Emirates,total,grocery/pharmacy,0,2020-02-20,-26,0.851749,784
...,...,...,...,...,...,...,...,...,...
207650,ZW,Zimbabwe,total,workplace,1,2020-04-01,-36,-70.065087,716
207651,ZW,Zimbabwe,total,workplace,1,2020-04-02,-36,-70.249640,716
207652,ZW,Zimbabwe,total,workplace,1,2020-04-03,-36,-67.565660,716
207653,ZW,Zimbabwe,total,workplace,1,2020-04-04,-36,-55.972998,716


In [30]:
from vega_datasets import data

world_topo = alt.topo_feature(data.world_110m.url, 'countries')

In [31]:
data.world_110m.url

'https://vega.github.io/vega-datasets/data/world-110m.json'

In [33]:
alt.data_transformers.enable('default')

categories = list(df.category.unique())
map_data = (df
              .query('region == "total"')
              .set_index(['country', 'country_code', 'region', 'category'])
            ['value']
            .div(100)
              .groupby(['country', 'country_code', 'region', 'category'])
              .last()
            .unstack()
              .reset_index())

input_dropdown = alt.binding_select(options=categories)
selection_category = alt.selection_single(fields=['category'], 
                                          bind=input_dropdown, 
                                          name='Mobility',
                                          init={'category': 'workplace'})

selection_country = alt.selection_multi(
        fields=['country'],
        name='Country of',
        empty='all',
#     init={'country': 'France'}
    )

background = (alt.Chart(world_topo)
              .mark_geoshape(fill='lightgray', stroke='white', strokeWidth=0.5)
              .transform_filter('datum.id != 10')
#               .transform_filter('datum.id != 304')
             )

foreground = (alt.Chart(world_topo)
        .mark_geoshape(stroke='white', strokeWidth=0.5)
         .encode(
             color=alt.condition(
                 selection_country,
                 alt.Color('value:Q', scale=alt.Scale(scheme='blueorange', domainMid=0), legend=alt.Legend(format=".0%")),
                 alt.value('lightgray')
             ),
             tooltip=[alt.Tooltip('value:Q', format='.0%'), 'country:N']
         )
            .transform_lookup(
                lookup='id',
                from_=alt.LookupData(data=map_data,
                                 key='country_code', 
                                 fields=['country'] + categories)
            )
            .transform_fold(
                fold=categories,
                as_=['category', 'value']
            )
             .add_selection(selection_category)
             .transform_filter(selection_category)
            ).add_selection(selection_country)

map_chart = (background + foreground).properties(width=700, height=500, 
#                                                  title="Variation to baseline on March 29"
                                                )

ts_data = df.query("region == 'total'").assign(value=lambda f: f['value'].div(100))
base_ts = alt.Chart(ts_data)

ts_chart = (base_ts
            .mark_line(point=True)
            .encode(x='date:T', 
                    y=alt.Y('value:Q', axis=alt.Axis(format='%')), 
                    tooltip=[alt.Tooltip('date:T', format='%a, %b %e'), alt.Tooltip('value:Q', format='.1%')])
#             .properties(title='Variation through time')
         .add_selection(selection_category)
            .add_selection(selection_country)
            .transform_filter(selection_category)
            .transform_filter(selection_country)
            .transform_aggregate(
                value='mean(value)',
                groupby=['category', 'date']
            )
        )

chart = (ts_chart | map_chart).properties(title='Mobility change by geography, across different categories of places (Variation to baseline as of April 5)')
chart.save('dist/static/mobility/charts/map.html')
alt.data_transformers.enable('data_server')
chart

In [15]:
subset = (df.query('country == "France" and region == "total"')
          .assign(date=lambda f: pd.to_datetime(f['date']))
          .assign(weekday=lambda f: f['date'].dt.dayofweek, 
                  weekofyear=lambda f: f['date'].dt.weekofyear,
                  is_weekend=lambda f: f['weekday'].isin([5, 6]).astype(int).astype(str),
                  time=lambda f: f['weekofyear'].astype(str).str.pad(2, fillchar='0').str.cat(f['is_weekend'])
                 )
)
subset

,country_geoid,region,category,page,change,date,value,country,country_code,weekday,weekofyear,is_weekend,time
51976,FR,total,retail/recreation,0,-88,2020-02-16,5.602760,France,250,6,7,1,071
51977,FR,total,retail/recreation,0,-88,2020-02-17,1.075916,France,250,0,8,0,080
51978,FR,total,retail/recreation,0,-88,2020-02-18,4.737228,France,250,1,8,0,080
51979,FR,total,retail/recreation,0,-88,2020-02-19,-3.767704,France,250,2,8,0,080
51980,FR,total,retail/recreation,0,-88,2020-02-20,-1.635872,France,250,3,8,0,080
...,...,...,...,...,...,...,...,...,...,...,...,...,...
52229,FR,total,residential,1,18,2020-03-25,31.331890,France,250,2,13,0,130
52230,FR,total,residential,1,18,2020-03-26,33.617282,France,250,3,13,0,130
52231,FR,total,residential,1,18,2020-03-27,36.963806,France,250,4,13,0,130
52232,FR,total,residential,1,18,2020-03-28,24.261902,France,250,5,13,1,131


In [16]:
(subset
 .groupby(['category', 'weekofyear'])
 .mean()
 .reset_index()
 .pipe(alt.Chart)
 .mark_line()
 .encode(x='weekofyear', y='value', color='category')
)